In [ ]:
%run block_swipdg.ipynb

In [ ]:
import numpy as np

In [ ]:
print('estimating error ...')

lambda_bar = lambda_
lambda_hat = lambda_
alpha_mu_mu_bar = 1.
gamma_mu_mu_bar = 1.
alpha_mu_mu_hat = 1.

local_sizes = [block_space.local_space(ii).size() for ii in range(block_space.num_blocks)]
local_starts = [int(np.sum(local_sizes[:ii])) for ii in range(block_space.num_blocks)]
local_starts.append(block_space.mapper.size)
localized_u_h_np = np.array(u_h_vector, copy=False)
localized_u_h_np = [localized_u_h_np[local_starts[ii]:local_starts[ii+1]] for ii in range(block_space.num_blocks)]
localized_u_h = [Vector(block_space.local_space(ii).size(), 0.) for ii in range(block_space.num_blocks)]
for ii in range(block_space.num_blocks):
    tmp = np.array(localized_u_h[ii], copy=False)
    tmp[:] = localized_u_h_np[ii][:]
local_boundary_info = make_subdomain_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.boundarysegmentindexbased',
                                                          'default': 'dirichlet',
                                                          'neumann': '[{} {}]'.format(inner_boundary_id,
                                                                                      inner_boundary_id+1)})

In [ ]:
print('  nonconformity indicator: ', end='')

# the local indicators
from dune.gdt import (
    make_ESV2007_nonconformity_product_dd_subdomain_part_dd_subdomain_oversampled_part
        as make_local_nonconformity_product
)

eta_nc_squared = 0.

for ii in range(grid.num_subdomains):
    neighborhood = grid.neighborhood_of(ii)
    neighborhood_space = block_space.restricted_to_neighborhood(neighborhood)

    def localize_to_subdomain_with_neighborhood_support(ss):
        return make_discrete_function(neighborhood_space,
                                      neighborhood_space.project_onto_neighborhood(
                                          [localized_u_h[nn] if nn == ss else Vector(block_space.local_space(nn).size(), 0.)
                                           for nn in neighborhood],
                                          neighborhood))
    subdomain_uh_with_neighborhood_support = {nn: localize_to_subdomain_with_neighborhood_support(nn)
                                              for nn in neighborhood}

    for jj in neighborhood:
        for kk in neighborhood:
            eta_nc_squared += make_local_nonconformity_product(grid, ii, ii, local_boundary_info,
                                                               lambda_hat, kappa,
                                                               subdomain_uh_with_neighborhood_support[jj],
                                                               subdomain_uh_with_neighborhood_support[kk],
                                                               over_integrate=2).apply2()

print('{} (should be 1.66e-01)'.format(np.sqrt(eta_nc_squared)))

In [ ]:
print('  residual indicator (RS2017): ', end='')

from dune.gdt import (
    make_rt_leaf_view_to_2x1_pdelab_p0_space as make_rt_space,
    apply_diffusive_flux_reconstruction_operator,
    RS2017_apply_l2_product as apply_l2_product,
    RS2017_residual_indicator_min_diffusion_eigenvalue as min_diffusion_eigenvalue,
    RS2017_residual_indicator_subdomain_diameter as subdomain_diameter
)

eta_r_RS2017_squared = 0.
global_rt_space = make_rt_space(grid)

for ii in range(grid.num_subdomains):
    neighborhood = grid.neighborhood_of(ii)
    neighborhood_space = block_space.restricted_to_neighborhood(neighborhood)

    # The following is not optimal, but the pdelab-based RT space does not work on the dd_subdomain_oversampled
    # grid view modeling the neighborhood (at least is the underlying grid is a 2d simplex alugrid) and we thus
    # need to reconstruct on the full leaf view.
    def localize_to_subdomain_with_global_support(ss):
        return make_discrete_function(
            block_space,
            block_space.project_onto_neighborhood(
                [localized_u_h[nn] if nn == ss else Vector(block_space.local_space(nn).size(), 0.)
                 for nn in range(grid.num_subdomains)],
                set([nn for nn in range(grid.num_subdomains)])))
    subdomain_uhs_with_global_support = {nn: localize_to_subdomain_with_global_support(nn)
                                         for nn in neighborhood}

    eta_r_RS2017_ii_squared = 0.
    # eta r (RS2017), f x f
    eta_r_RS2017_ii_squared += apply_l2_product(grid, ii, f, f, over_integrate=2)
    # eta r (RS2017), f x u
    for jj in neighborhood:
        reconstructed_uh_jj_with_global_support = make_discrete_function(global_rt_space)
        apply_diffusive_flux_reconstruction_operator(lambda_, kappa,
                                                     subdomain_uhs_with_global_support[jj],
                                                     reconstructed_uh_jj_with_global_support)
        eta_r_RS2017_ii_squared -= 2.*apply_l2_product(grid, ii,
                                                       f,
                                                       reconstructed_uh_jj_with_global_support.divergence(),
                                                       over_integrate=2)
    # eta r (RS2017), u x u
    for jj in neighborhood:
        reconstructed_uh_jj_with_global_support = make_discrete_function(global_rt_space)
        apply_diffusive_flux_reconstruction_operator(lambda_, kappa,
                                                     subdomain_uhs_with_global_support[jj],
                                                     reconstructed_uh_jj_with_global_support)
        for kk in neighborhood:
            reconstructed_uh_kk_with_global_support = make_discrete_function(global_rt_space)
            apply_diffusive_flux_reconstruction_operator(lambda_, kappa,
                                                         subdomain_uhs_with_global_support[kk],
                                                         reconstructed_uh_kk_with_global_support)
            eta_r_RS2017_ii_squared += apply_l2_product(grid, ii,
                                                        reconstructed_uh_jj_with_global_support.divergence(),
                                                        reconstructed_uh_kk_with_global_support.divergence(),
                                                        over_integrate=2)
    # eta r (RS2017), scale
    poincaree_constant = 1./(np.pi**2)
    min_diffusion_ev = min_diffusion_eigenvalue(grid, ii, lambda_hat, kappa)
    subdomain_h = subdomain_diameter(grid, ii)
    eta_r_RS2017_ii_squared *= (poincaree_constant/min_diffusion_ev) * subdomain_h**2
    eta_r_RS2017_squared += eta_r_RS2017_ii_squared

print('{} (should be 2.89e-01)'.format(np.sqrt(eta_r_RS2017_squared)))

In [ ]:
print('  diffusive flux indicator (RS2017): ', end='')

from dune.gdt import (
    RS2017_diffusive_flux_indicator_apply_aa_product as apply_diffusive_flux_aa_product,
    RS2017_diffusive_flux_indicator_apply_ab_product as apply_diffusive_flux_ab_product,
    RS2017_diffusive_flux_indicator_apply_bb_product as apply_diffusive_flux_bb_product
)

eta_df_RS2017_squared = 0.

for ii in range(grid.num_subdomains):
    neighborhood = grid.neighborhood_of(ii)
    neighborhood_space = block_space.restricted_to_neighborhood(neighborhood)

    # The following is not optimal, see above.
    def localize_to_subdomain_with_global_support(ss):
        return make_discrete_function(
            block_space,
            block_space.project_onto_neighborhood(
                [localized_u_h[nn] if nn == ss else Vector(block_space.local_space(nn).size(), 0.)
                 for nn in range(grid.num_subdomains)],
                set([nn for nn in range(grid.num_subdomains)])))
    subdomain_uhs_with_global_support = {nn: localize_to_subdomain_with_global_support(nn)
                                         for nn in neighborhood}
    
    subdomain_uh = make_discrete_function(block_space.local_space(ii), localized_u_h[ii])

    eta_df_RS2017_ii_squared = 0.
    # eta df (RS2017), u x u
    eta_df_RS2017_ii_squared += apply_diffusive_flux_aa_product(grid, ii,
                                                                lambda_hat, lambda_u=lambda_, lambda_v=lambda_,
                                                                kappa=kappa,
                                                                u=subdomain_uh,
                                                                v=subdomain_uh,
                                                                over_integrate=2)
    # eta df (RS2017), u x reconstructed_u
    for jj in neighborhood:
        reconstructed_uh_jj_with_global_support = make_discrete_function(global_rt_space)
        apply_diffusive_flux_reconstruction_operator(lambda_, kappa,
                                                     subdomain_uhs_with_global_support[jj],
                                                     reconstructed_uh_jj_with_global_support)
        eta_df_RS2017_ii_squared += 2.*apply_diffusive_flux_ab_product(
            grid, ii,
            lambda_hat, lambda_u=lambda_, kappa=kappa,
            u=subdomain_uh,
            reconstructed_v=reconstructed_uh_jj_with_global_support,
            over_integrate=2)
    # eta df (RS2017), reconstructed_u x reconstructed_u
    for jj in neighborhood:
        reconstructed_uh_jj_with_global_support = make_discrete_function(global_rt_space)
        apply_diffusive_flux_reconstruction_operator(lambda_, kappa,
                                                     subdomain_uhs_with_global_support[jj],
                                                     reconstructed_uh_jj_with_global_support)
        for kk in neighborhood:
            reconstructed_uh_kk_with_global_support = make_discrete_function(global_rt_space)
            apply_diffusive_flux_reconstruction_operator(lambda_, kappa,
                                                         subdomain_uhs_with_global_support[kk],
                                                         reconstructed_uh_kk_with_global_support)
            eta_df_RS2017_squared += apply_diffusive_flux_bb_product(
                grid, ii,
                lambda_hat, kappa,
                reconstructed_uh_jj_with_global_support,
                reconstructed_uh_kk_with_global_support,
                over_integrate=2)

    eta_df_RS2017_squared += eta_df_RS2017_ii_squared

print('{} (should be 3.55e-01)'.format(np.sqrt(eta_df_RS2017_squared)))